# E-commerce Customer Profiling and Data Consolidation Project

# Problem Statement

An e-commerce startup, after a successful first month, is seeking to better understand its customer base and purchasing patterns. They aim to answer fundamental questions like "Who are our customers?" "What do they buy?" and "What drives their purchasing behavior?" However, they face challenges in accurately identifying and counting customers due to disparate data sources, including:

1. A customer database recording online account sign-ups,
2. A CRM system tracking phone and non-online customer interactions, and
3. Raw transaction data that includes guest purchases with no formal customer record.

These sources of data may have overlapping entries or duplicate records, often due to customers engaging with multiple systems (e.g., making a purchase as a guest, then creating an account later). Furthermore, customer information may vary across entries due to potential discrepancies like typos or alternative identifiers. The startup seeks the assistance of an analyst to address these complexities and provide an accurate count and profile of their customers.


## Project Goals

1. **Data Consolidation:**  
   Integrate customer information across all available sources to create a unified, comprehensive dataset representing the complete customer base.

2. **Data Deduplication:**  
   Identify and resolve duplicate records, maintaining data lineage to track original sources and identifiers for each customer where possible.

3. **Customer Identification:**  
   Develop a data model that uniquely identifies each customer, regardless of which data source(s) they appear in, and can be easily queried for counts and profiles.

4. **Data Validation:**  
   Ensure data integrity by confirming assumptions, identifying inconsistencies, and validating data completeness to maximize reliability.

5. **Solution Documentation:**  
   Create a flexible and traceable solution, with a well-defined schema that supports future analysis of customer demographics and purchasing behavior.

This project will ultimately enable the startup to answer critical questions about their customer demographics and purchasing trends, creating a foundation for data-driven decision-making.

To actually perform the data modeling, we should take the following course of action:

1. First we explore all 3 datasets. We should see if the values in all the columns make sense.

2. Then we can trim down our data before we merge our different datasets, so that we have unique customers

3. Then we merge these trimmed datasets and remove duplicates carefully.

4. When performing deduplication, we need to go one step beyond and identify more than just exact match duplicates. We can use Fuzzy Matching here.

5. Finally, we clean and present our data model.

## Exploring our Purchases Dataset

In [5]:
import pandas as pd
import numpy as np

sales = pd.read_csv("../data/purchases.csv")
print(sales.shape)

(71519, 11)


In [6]:
sales.head()


,event_time,product_id,category_id,category_code,brand,price,session_id,customer_id,guest_first_name,guest_surname,guest_postcode
0,2022-10-01 02:26:08+00:00,32701106,2055156924466332447,NaN,shimano,95.21,64c68405-7002-4ce0-9604-a4c2e1f7384b,NaN,MICHAEL,MASON,RG497ZQ
1,2022-10-01 02:28:32+00:00,9400066,2053013566067311601,NaN,jaguar,164.20,3b7d6741-3c82-4c75-8015-6f54b52612e0,7466.0,NaN,NaN,NaN
2,2022-10-01 02:31:01+00:00,1004238,2053013555631882655,electronics.smartphone,apple,1206.40,38c6d3f7-6c32-4fed-bca6-ef98e1746386,NaN,COLE,WILKINSON,SW75TQ
3,2022-10-01 02:33:31+00:00,11300059,2053013555531219353,electronics.telephone,texet,17.48,3398c966-7846-4186-89be-323daad735b9,NaN,MOHAMMED,RICHARDS,RG150RE
4,2022-10-01 02:40:18+00:00,17300751,2053013553853497655,NaN,versace,77.22,11e3a573-01b9-4794-b513-e7d8a4fcac83,31266.0,NaN,NaN,NaN


In [7]:
sales.isnull().sum()

event_time              0
product_id              0
category_id             0
category_code       16739
brand                5707
price                   0
session_id              0
customer_id         18448
guest_first_name    53071
guest_surname       53071
guest_postcode      53071
dtype: int64

Looks like the 18448 missing customer IDs (which are guest checkouts) and the 53071 missing guest values (which are registered customers) make up all of our data. We should verify there's no overlap, e.g. a row with both customer ID and guest details missing.

Let's create a column to track guest checkouts:

In [8]:
sales["is_guest"] = sales["customer_id"].isnull()

Let's check cases where a guest checkout also had a customer id filled

In [9]:
sales[sales["is_guest"] & sales["customer_id"].notnull()]

,event_time,product_id,category_id,category_code,brand,price,session_id,customer_id,guest_first_name,guest_surname,guest_postcode,is_guest


no such case. Let's also check cases where we neither have a guest checkout nor a customer id filled.

In [10]:
sales[(sales["is_guest"] == False) & sales["customer_id"].isnull()]

,event_time,product_id,category_id,category_code,brand,price,session_id,customer_id,guest_first_name,guest_surname,guest_postcode,is_guest
